In [10]:
# Importing necessary libraries
import os
import pandas as pd
from dotenv import load_dotenv
from googleapiclient.discovery import build

# Load environment variables from a .env file
load_dotenv() 

# Define YouTube API key and version
API_KEY = '' # Put your Google API key here
API_VERSION = 'v3'

# Build the YouTube API client
youtube = build('youtube', API_VERSION, developerKey=API_KEY)

# Function to get the number of subscribers, views, and videos of each channel
def get_channel_stats(youtube, channel_id):
    # Create a request to get channel statistics
    request = youtube.channels().list(
        part='snippet, statistics',
        id=channel_id
    )
    # Execute the request and get the response
    response = request.execute()
    # Check if the response contains data
    if 'items' in response and response['items']:
        # Extract the relevant data from the response
        data = dict(channel_name=response['items'][0]['snippet']['title'],
                    total_subscribers=response['items'][0]['statistics']['subscriberCount'],
                    total_views=response['items'][0]['statistics']['viewCount'],
                    total_videos=response['items'][0]['statistics']['videoCount'],
        )
        return data
    else:
        print(f"No data found for channel ID: {channel_id}") 
        return None

# Example channel ID to test the function
channel_id = "UC9LQwHZoucFT94I2h6JOcjw"
print(get_channel_stats(youtube, channel_id))

# Read CSV file into a dataframe
df = pd.read_csv("youtube_data_mexico.csv")

# Extract channel IDs from the dataframe and remove potential duplicates
channel_ids = df['NAME'].str.split('@').str[-1].unique()

# Initialise a list to store channel statistics
channel_stats = []

# Loop over the channel IDs and get stats for each channel
for channel_id in channel_ids:
    stats = get_channel_stats(youtube, channel_id)
    if stats is not None:
        channel_stats.append(stats)

# Convert the list of channel stats to a dataframe
stats_df = pd.DataFrame(channel_stats)

# Reset index for consistency
df.reset_index(drop=True, inplace=True)
stats_df.reset_index(drop=True, inplace=True)

# Concatenate the original dataframe with the stats dataframe horizontally
combined_df = pd.concat([df, stats_df], axis=1)

# Optionally, drop the 'channel_name' column from stats_df (since 'NAME' already exists in the original dataframe) 
# # I preferred to keep it to review any discrepancies or missing data 
# # combined_df.drop('channel_name', axis=1, inplace=True)

# Save the merged dataframe back into a CSV file
combined_df.to_csv('updated_youtube_data_mex.csv', index=False)

# Display the first 10 rows of the combined dataframe
combined_df.head(10)

{'channel_name': 'Liverpool FC', 'total_subscribers': '10900000', 'total_views': '3199123107', 'total_videos': '7605'}
No data found for channel ID: mispastelitos
No data found for channel ID: CRACKSYOUTUBE


,#,NAME,FOLLOWERS,ER,COUNTRY,TOPIC OF INFLUENCE,POTENTIAL REACH,channel_name,total_subscribers,total_views,total_videos
0,1,Badabun @UCYWOjHweP2V-8kGKmmAmQJQ,45.7M,-,Mexico,NaN,13.7M,Badabun,47600000,19911915821,24080
1,2,Kimberly Loaiza @UCQZfFRohQ7UX-0CdXl-6pwQ,38.1M,0.90%,Mexico,NaN,11.4M,Kimberly Loaiza,46300000,6943011571,314
2,3,GENIAL @UCbrd1vu4_7qIE6IPV_dA-OA,31.2M,-,Mexico,NaN,9.4M,GENIAL,32000000,7081434709,9199
3,4,Masha y el Oso @UCuSo4gcgxJRf4Bzu43wwVyg,25.9M,-,Mexico,NaN,7.8M,Masha y el Oso,39600000,20262996461,1667
4,5,MrBeast en Español @UCNAhW2oJvy_FUU082pEDspA,22M,1.80%,Mexico,NaN,6.6M,MrBeast en Español,26400000,3148202215,67
5,6,BabyBus - Canciones Infantiles & Cuentos @UCy_...,21.3M,-,Mexico,NaN,6.4M,BabyBus - Canciones Infantiles & Videos para N...,26800000,15841426193,1674
6,7,Juan De Dios Pantoja @UCzoUWqjCbcfWFdOMvoep8FA,20.7M,1.30%,Mexico,NaN,6.2M,Juan De Dios Pantoja,44000000,5857439622,432
7,8,Tlnovelas @UCKyU-wd-KY4PMOcOpPQgQcw,20.1M,-,Mexico,NaN,6M,Tlnovelas,24800000,19216815453,59898
8,9,YOLO AVENTURAS @UCOnIJiQuk1fDSp6p1GCZy3A,17.9M,0.60%,Mexico,NaN,5.4M,YOLO AVENTURAS,58500000,20524794143,2699
9,10,YOLO @UCQ8TuCvcDMepleXFyOQfyOQ,17.9M,-,Mexico,NaN,5.4M,YOLO,31900000,2488960701,362
